In [9]:
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D, Input, Activation
from keras.layers import LSTM, CuDNNLSTM
from keras.layers import TimeDistributed
from keras.layers import Conv2D, MaxPooling3D, Conv3D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
from keras.regularizers import l2
from collections import deque
import sys

class AVAnalysisModel():   
    def __init__(self, seq_length, saved_model=None):
        self.seq_length = seq_length
        self.input_shape = (seq_length, 80, 80, 3)
        
        if saved_model is not None:
            self.model = load_model(self.saved_model)
        else:
            self.model = self.lrcn()
            
        # Now compile the network.
        optimizer = Adam(lr=1e-5, decay=1e-6)
        self.model.compile(loss='categorical_crossentropy', optimizer='adam',
                           metrics=['accuracy'])

    
    def lrcn(self):
        """Build a CNN into RNN.
        Starting version from:
            https://github.com/udacity/self-driving-car/blob/master/
                steering-models/community-models/chauffeur/models.py
        Heavily influenced by VGG-16:
            https://arxiv.org/abs/1409.1556
        Also known as an LRCN:
            https://arxiv.org/pdf/1411.4389.pdf
        """
        def add_default_block(model, kernel_filters, init, reg_lambda):

            # conv
            model.add(TimeDistributed(Conv2D(kernel_filters, (3, 3), padding='same',
                                             kernel_initializer=init, kernel_regularizer=l2(l=reg_lambda))))
            model.add(TimeDistributed(BatchNormalization()))
            model.add(TimeDistributed(Activation('relu')))
            # conv
            model.add(TimeDistributed(Conv2D(kernel_filters, (3, 3), padding='same',
                                             kernel_initializer=init, kernel_regularizer=l2(l=reg_lambda))))
            model.add(TimeDistributed(BatchNormalization()))
            model.add(TimeDistributed(Activation('relu')))
            # max pool
            model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

            return model

        initialiser = 'glorot_uniform'
        reg_lambda  = 0.001

        model = Sequential()

        # first (non-default) block
        model.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2), padding='same',
                                         kernel_initializer=initialiser, kernel_regularizer=l2(l=reg_lambda)),
                                  input_shape=self.input_shape))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Activation('relu')))
        model.add(TimeDistributed(Conv2D(32, (3,3), kernel_initializer=initialiser, kernel_regularizer=l2(l=reg_lambda))))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(Activation('relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        # 2nd-5th (default) blocks
        #model = add_default_block(model, 64,  init=initialiser, reg_lambda=reg_lambda)
        #model = add_default_block(model, 128, init=initialiser, reg_lambda=reg_lambda)
        #model = add_default_block(model, 256, init=initialiser, reg_lambda=reg_lambda)
        #model = add_default_block(model, 512, init=initialiser, reg_lambda=reg_lambda)
        model = add_default_block(model, 92, init=initialiser, reg_lambda=reg_lambda)
        model = add_default_block(model, 196, init=initialiser, reg_lambda=reg_lambda)
        
        # LSTM output head
        model.add(TimeDistributed(Flatten()))
        model.add(CuDNNLSTM(256, return_sequences=False))
        model.add(Dense(len(VideoClass), activation='softmax'))

        return model
    
    def load_model(allow_load=True):
        models = build_models()
        models[0].summary()
        if allow_load:
            try:
                models[0].load_weights(MODEL_FILE)
                print('Loaded model from file.')
            except:
                print('Unable to load model from file.')
        return models

In [10]:
import time
import os.path
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from video_analysis_constants import *
from video_dataset import *


def train(seq_length, saved_model=None, image_shape=None,
          load_to_memory=False, batch_size=32, nb_epoch=100):
    
    # Helper: TensorBoard
    tb = TensorBoard(log_dir=os.path.join(OUT_FOLDER, 'logs'))

    # Helper: Stop when we stop learning.
    early_stopper = EarlyStopping(patience=5)
    
    #Helper: Save the model during training
    checkpointer = ModelCheckpoint(
        filepath=MODEL_FILE,
        monitor='model',
        verbose=1,
        save_best_only=True)

    if image_shape is None:
        data = DataSet(
            seq_length=seq_length
        )
    else:
        data = DataSet(
            seq_length=seq_length,
            image_shape=image_shape
        )

    # Get samples per epoch.
    # Multiply by PERCENT_OF_TRAIN to attempt to guess how much of data.data is the train set.
    steps_per_epoch = (len(data.data) * PERCENT_OF_TRAIN) // batch_size

    if load_to_memory:
        # Get data.
        X, y, X_test, y_test = data.get_all_sequences_in_memory(PERCENT_OF_TRAIN)
    else:
        # Get generators.
        #generator = data.frame_generator(batch_size, 'train', PERCENT_OF_TRAIN)
        #val_generator = data.frame_generator(batch_size, 'test', PERCENT_OF_TRAIN)
        generator, val_generator = data.get_generators(batch_size, PERCENT_OF_TRAIN)

    # Get the model.
    rm = AVAnalysisModel(seq_length, saved_model)
    

    # Fit!
    if load_to_memory:
        # Use standard fit.
        rm.model.fit(
            X,
            y,
            batch_size=batch_size,
            validation_data=(X_test, y_test),
            verbose=1,
            callbacks=[tb, early_stopper, checkpointer],
            epochs=nb_epoch)
    else:
        # Use fit generator.
        rm.model.fit_generator(
            generator=generator,
            steps_per_epoch=steps_per_epoch,
            epochs=nb_epoch,
            verbose=1,
            callbacks=[tb, early_stopper, checkpointer],
            validation_data=val_generator,
            validation_steps=40,
            workers=4)
 
    now = time.strftime("%c")
    rm.save(str(now) + '.model')

In [11]:
saved_model = None  # None or weights file
seq_length = 40
batch_size = 32
nb_epoch = 1000
image_shape = (80, 80, 3)


In [ ]:
load_to_memory=False
train(seq_length, saved_model=saved_model, image_shape=image_shape, load_to_memory=load_to_memory, batch_size=batch_size, nb_epoch=nb_epoch)